In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("BMI.csv")[["Height","Weight","Index"]]
df

,Height,Weight,Index
0,174,96,4
1,189,87,2
2,185,110,4
3,195,104,3
4,149,61,3
...,...,...,...
495,150,153,5
496,184,121,4
497,141,136,5
498,150,95,5


df["Gender"]=df["Gender"].map({"Male":1,"Female":0})
df

In [3]:
x=df.iloc[:,:-1]
x
y=df.iloc[:,-1]
y

0      4
1      2
2      4
3      3
4      3
      ..
495    5
496    4
497    5
498    5
499    5
Name: Index, Length: 500, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=100)

In [6]:
x_test.shape

(150, 2)

In [7]:
model=LinearRegression()

In [8]:
model.fit(x_train,y_train)

LinearRegression()

In [9]:
y_pred=model.predict(x_test)
y_pred

array([3.90303164, 4.94565292, 4.48191492, 4.03241904, 3.17402952,
       3.78340752, 5.74810676, 3.04699688, 2.05238637, 3.81642912,
       4.10098912, 1.81239107, 4.81230307, 2.23409124, 3.85846694,
       5.64525163, 4.30939839, 4.12752139, 2.50980716, 4.87960971,
       0.9946038 , 4.61652821, 5.16778791, 4.04631689, 3.2577609 ,
       2.38639271, 4.06308588, 4.99888958, 3.66774585, 1.67777778,
       3.6502298 , 2.77237227, 1.4745944 , 3.23484683, 2.81205534,
       3.67819763, 4.507528  , 2.2595322 , 3.76807409, 4.06147819,
       3.17638427, 3.86461202, 3.77312786, 2.75611967, 4.22945733,
       2.7546841 , 1.77557914, 3.54341222, 3.36928799, 2.6619365 ,
       4.34098442, 2.82468975, 2.82216287, 3.75055804, 4.02610183,
       2.90968457, 2.45369936, 1.65629927, 5.78112836, 3.94489733,
       3.67946107, 3.53944976, 4.69267894, 4.96839486, 2.65182897,
       4.54560337, 4.6583939 , 4.1998818 , 5.19070198, 4.03603723,
       3.47231525, 2.21513962, 2.90210393, 4.44004923, 5.04598

In [10]:
model.coef_

array([-0.03428504,  0.0330216 ])

In [11]:
from sklearn import metrics

In [12]:
print("R2 score for testing",metrics.r2_score(y_test,y_pred))

R2 score for testing 0.838723547585764


In [13]:
y_pred_train=model.predict(x_train)

In [14]:
print("R2 score for traing",metrics.r2_score(y_train,y_pred_train))

R2 score for traing 0.8200684838760557


adjusted_r2=1-(1-r2)*(n-1)/(n-p-1)

In [15]:
def adj_r2(x,y):
    r2=model.score(x,y)
    n=x.shape[0]
    p=x.shape[1]
    adjusted_r2=1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2


print("Adj r2 for training data",adj_r2(x_train,y_train))

print("Adj r2 for testing data",adj_r2(x_test,y_test))

Adj r2 for training data 0.8190314146188571
Adj r2 for testing data 0.8365293101379512


In [16]:
print("mean_squared_error",metrics.mean_squared_error(y_test,y_pred))
print("mean_absolute_error",metrics.mean_absolute_error(y_test,y_pred))
print("mean_absolute_error",metrics.mean_absolute_percentage_error(y_test,y_pred))


mean_squared_error 0.28510809649905117
mean_absolute_error 0.43753961255282275
mean_absolute_error 111051066615960.11


In [17]:
import numpy as np

print("Accuracy of the model",100-(np.median(np.abs((y_test-y_pred)/y_test))*100))

Accuracy of the model 89.72923611931614


In [18]:
result=model.predict(np.array([[150,100]]))


C:\Users\mohit\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [19]:
result[0]

4.222795870547438

In [20]:
import pickle
pickle.dump(model,open("model.pkl","wb"))

BMI 

import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('BMI.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    initial_features = [int(x) for x in request.form.values()]
    final_feature = [np.array(initial_features)]
    prediction = model.predict(final_feature)

    output = round(prediction[0], 2)

    return render_template('BMI.html', prediction_text='BMI of the person is {}'.format(output))




if __name__ == "__main__":
    app.run(debug=True)

house price

from flask import Flask,render_template,request
import pickle
import numpy as np
app = Flask(__name__)
model=pickle.load(open('model.pkl','rb'))

@app.route('/')
def home():
    return render_template('BMI.html')

@app.route('/predict',methods=["POST"])
def predict():
    feature=[int(x) for x in request.form.values()]
    feature_final=[np.array(feature)]
    prediction=model.predict(feature_final)
    output=round(prediction[0],1)
    return render_template('BMI.html',prediction_text='BMI of the person is. {}'.format(output))

if __name__=="__main__":
    app.run(debug=True)



BMI

<!DOCTYPE html>
<html >
<!--From https://codepen.io/frytyler/pen/EGdtg-->
<head>
  <meta charset="UTF-8">
  <title>ML API</title>
  <link href='https://fonts.googleapis.com/css?family=Pacifico' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Arimo' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Hind:300' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Open+Sans+Condensed:300' rel='stylesheet' type='text/css'>
<link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">
  
</head>

<body>
 <div class="login">
	<h1> BMI Prediction</h1>

     <!-- Main Input For Receiving Query to our ML -->
    <form action="{{ url_for('predict')}}"method="post">
    	<input type="text" name="height" placeholder="Height" required="required" />
        <input type="text" name="Weight" placeholder="Weight" required="required" />
		

        <button type="submit" class="btn btn-primary btn-block btn-large">Predict</button>
    </form>

   <br>
   <br>
   {{ prediction_text }}

 </div>


</body>
</html>


house price

<html>
<head>
<style>
.button {
background-color: #23b83c;
	border: 2px solid #020353;
	color: whitesmoke;
	padding: 0.5em 0.5em;
	text-align: center;
	text-decoration: none;
	font-size: 1em;
	display: inline-block;
	width: 18%;
	border-radius: 5px;
	box-shadow: 3px 3px 8px 0 #000; 
}
body {
  background-image: url("{{url_for('static', filename='background.jpg')}}");
  background-repeat: no-repeat;
 background-size: 100% 100%;
}
h1{
   background-color: Green;
  color: white;
}
h3{
   background-color: green;
  color: white;
}


</style>
</head>
<body>
<center>


<h1>BMI Prediction</h1>
<pre>
<form action="{{ url_for('predict')}}"method="post">
<input type="text" name="height" placeholder="Height" required="required" />
<input type="text" name="Weight" placeholder="Weight" required="required" />



<button type="submit" class="button">Predict</button>
</form>
<textarea>
{{prediction_text}}
</textarea>
</pre>
</center>
</body>
</html>